In [8]:
import numpy as np
import torch
import gdown
import os

def download_dataset():
    file_id = "19MBh9JIJTxYIPAeO7G5RML5_ddjJ1Cpa"
    output_path = "downloaded.zip"

    if not os.path.exists(output_path):
        print(f"File '{output_path}' not found. Downloading...")
        gdown.download(f"https://drive.google.com/uc?export=download&id={file_id}", output_path, quiet=False)
        print("Download complete!")
    else:
        print(f"File '{output_path}' already exists. Skipping download.")

download_dataset()



File 'downloaded.zip' already exists. Skipping download.
